# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
!pip install citipy

In [4]:
!pip install geoviews

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pangai,-19.8000,-174.3500,23.06,56,20,4.63,TO,1727378987
1,1,west island,-12.1568,96.8225,25.99,83,100,4.63,CC,1727378989
2,2,anloga,5.7947,0.8973,25.82,85,17,4.64,GH,1727378990
3,3,papatowai,-46.5619,169.4708,3.98,76,96,6.89,NZ,1727378991
4,4,chibougamau,49.9168,-74.3659,14.79,94,100,3.60,CA,1727378992


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
)

# Display the map
map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27)]
city_data_df = city_data_df.loc[city_data_df["Wind Speed"] < 4.5]
city_data_df = city_data_df.loc[city_data_df["Cloudiness"] == 0]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
89,89,creston,41.0586,-94.3614,25.98,36,0,2.06,US,1727379092
125,125,belmonte,-15.8631,-38.8828,24.80,70,0,3.42,BR,1727379134
194,194,afaahiti,-17.7500,-149.2833,25.07,80,0,3.43,PF,1727379212
209,209,caucete,-31.6518,-68.2811,22.46,16,0,3.56,AR,1727379213
248,248,winnemucca,40.9730,-117.7357,26.03,17,0,0.00,US,1727379274


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns_copy = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = city_data_df[columns_copy].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
89,creston,US,41.0586,-94.3614,36,
125,belmonte,BR,-15.8631,-38.8828,70,
194,afaahiti,PF,-17.7500,-149.2833,80,
209,caucete,AR,-31.6518,-68.2811,16,
248,winnemucca,US,40.9730,-117.7357,17,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
category = "accommodation.hotel"

params = {
    "apiKey":geoapify_key,
    "categories":category
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] =  f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
   
    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
creston - nearest hotel: No hotel found
belmonte - nearest hotel: No hotel found
afaahiti - nearest hotel: Omati Lodge
caucete - nearest hotel: No hotel found
winnemucca - nearest hotel: The Overland Hotel
ar rutbah - nearest hotel: No hotel found
pacific grove - nearest hotel: Pacific Grove Inn
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
salinas - nearest hotel: The Traveler's Hotel
jackson - nearest hotel: Brookside
mansourah - nearest hotel: Marshall
boise - nearest hotel: The Grove Hotel
antsohihy - nearest hotel: Escale
agat village - nearest hotel: Navy Getway Inns & Suites
beira - nearest hotel: Hotel Savoy


,City,Country,Lat,Lng,Humidity,Hotel Name
89,creston,US,41.0586,-94.3614,36,No hotel found
125,belmonte,BR,-15.8631,-38.8828,70,No hotel found
194,afaahiti,PF,-17.7500,-149.2833,80,Omati Lodge
209,caucete,AR,-31.6518,-68.2811,16,No hotel found
248,winnemucca,US,40.9730,-117.7357,17,The Overland Hotel
274,ar rutbah,IQ,33.0381,40.2848,31,No hotel found
279,pacific grove,US,36.6177,-121.9166,83,Pacific Grove Inn
314,hotan,CN,37.0997,79.9269,21,Hua Yi International Hotel (accepts foreigners)
325,salinas,US,36.6777,-121.6555,71,The Traveler's Hotel
345,jackson,US,42.4165,-122.8345,54,Brookside


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    color = "Country",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Hotel Name)